<a href="https://colab.research.google.com/github/tirals88/pytorch-drl/blob/main/DRL_Chap3_2_CartPole_Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'PyTorch를 활용한 강화학습/심층강화학습 실전입문' 책 스터디 내용을 Google Colab으로 정리하여 올립니다.

Github 예제 코드 주소 : 'https://github.com/wikibook/pytorch-drl'

PyTorch를 활용한 강화학습/심층강화학습 실전입문

https://wikibook.co.kr/pytorch-drl/

### ***가상환경 구현 및 함수 수정***

In [ ]:
# matplotlib -downgrade

!pip install matplotlib==3.4.2

In [ ]:
!apt update
!apt install -y xvfb #server install
import sys

IS_COLAB = "google.colab" in sys.modules

if IS_COLAB:
  !apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
  %pip install -U tf-agents pyvirtualdisplay
  %pip install -U gym>=0.21.0
  %pip install -U gym[box2d,atari, accept-rom-license]

!xvfb-run -s "-screen 0 1400x900x24" jupyter notebook

### ***display_frames_as_gif 함수 정의***

In [1]:
# 구현에 사용할 패키지 임포트
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [ ]:
# 애니메이션을 만드는 함수
# 참고 URL : https://github.com/patrickmineault/xcorr-notebooks/blob/master/notebooks/Render%20OpenAI%20gym%20as%20GIF.ipynb
!pip install JSAnimation
from JSAnimation.IPython_display import display_animation
from IPython.display import display

#새로 정의한 display_animtaion 함수

from IPython.display import HTML

def display_animation(anim):
  return HTML(anim.to_jshtml())

In [ ]:
def display_frames_as_gif(frames):
  """
  Displays a list of frames as a gif, with controls
  """
  plt.figure(figsize=(frames[0].shape[1]/48.0, frames[0].shape[0]/48.0), dpi=72)
  patch = plt.imshow(frames[0])
  plt.axis('off')

  def animate(i):
    patch.set_data(frames[i])

  anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=20)

  anim.save('movie_cartpole.mp4') #애니메이션을 저장하는 부분
  display(display_animation(anim)) #수정된 부분

## 3.3 다변수, 연속값 상태를 표형식으로 나타내기

### CartPole 의 상태

이전 장에서 다뤘던 미로 태스크에서 상태는 에이전트가 어느 칸에 위치했는지를 변수 하나로 나타냈으며 0~8의 단순 이산값이었다. 그러나 역진자 태스크에서는 더 복잡하게 상태가 정의된다.

CartPole 태스크는 observation 변수에 상태를 저장한다. env.step(action)은 게임 환경을 1단계 진행시키는 명령어로 action = 0 일 때 수레를 왼쪽으로 움직이는 것이고 1 일 때 오른쪽으로 움직이는 것에 해당한다.

env.step(action) 은 observation, reward, done, info 라는 4개의 변수를 반환한다. observation은 수레와 봉의 상태를 나타내며 상태는 다시 4개의 변수로 이루어져 있다. 이는 수레의 위치 (-2.4 ~ 2.4), 수레의 속도 (-inf ~ inf), 봉의 각도 (-41.8도 ~ 41.8), 봉의 가속도 (-inf ~ inf)이다.

reward는 즉각보상이며 action을 실행한 후에 수레의 위치가 $\pm2.4$ 범위 안에 있고 봉이 $20.9$도 이상 기울어 있지 않다면 보상 1을 받는다. 반대의 경우, 보상은 0이 된다.

done 은 게임 종료 여부를 나타내며, 200단계를 초과하거나, 보상이 0일 될 때 게임이 종료되어 done 의 값이 True가 된다.

info 는 디버깅 등에 필요한 정보를 담은 변수이다.

observation의 4가지 변수는 모두 연속값으로 이루어져있다. 이를 표형식으로 나타내기 위해 연속값을 이산값으로 변환해야 한다.

예를 들어 수레의 위치를 6개의 구간의 이산값으로 변환한다면 -2.4 / -1.6 / -0.8 / 0 / 0.8 / 1.6 / 2.4 로 구간을 나누어 0 ~ 5의 값을 반환할 수 있다. 그러나 이 양 끝 범위를 벗어날 가능성도 있으므로 -Inf ~ -1.6 을 0으로, 1.6 ~ Inf 를 5로 정의한다.

다른 변수도 6개의 구간을 갖는 이산변수로 변환한다. 그러면 변수의 가짓수가 $6^{4} = 1296$개로 수레의 상태를 나타낼 수 있다.

CartPole 에서 취할 수 있는 행동은 수레를 오른쪽이나 왼쪽으로 밀기 두 가지다. 이후 수레는 가속도를 부여받는다. 각 상태변수를 6개의 구간을 갖는 이산변수로 변환하면 CartPole의 Q함수는 1296행 2열로 된 표로 나타낼 수 있다.

### 상태의 이산변수 변환 구현



```
ENV = 'CartPole-v0' # 태스크 이름
NUM_DIZITIZED = 6 # 각 상태를 이산변수로 변환할 구간 수

env = gym.make(ENV) # 실행할 태스크 설정
observation = env.reset() # 환경 초기화
```

env.reset()을 호출하면 초기 상태가 반환되므로 이 반환값을 observation 에 저장한다. 그 다음 이 변수의 값을 이산변수로 변환하는 함수를 정의한다.



```
# 이산값으로 만들 구간 계산
def bins(climp_min, climp_max, num):
  '''
  관측된 상태(연속값)을 이산값으로 변환하는 구간을 계산
  '''
  return np.linspace(climp_min, climp_max, num+1)[1:-1]

def digitize_state(observation):
  '''
  관측된 상태(observation 변수)를 이산값으로 변환
  '''
  cart_pos, cart_v, pole_angle, pole_v = observation

  digitized = [
    np.digitize(cart_post, bins=bins(-2.4, 2.4, NUM_DIGITIZED)),
    np.digitize(cart_v, bins=bins(-3.0, 3.0, NUM_DIGITIZED)),
    np.digitize(pole_angle, bins=bins(-0.5, 0.5, NUM_DIGITIZED)),
    np.digitize(pole_v, bins=bins(-2.0, 2.0, NUM_DIGITIZED))]

  return sum([x * (NUM_DIGITIZED ** i) for i, x in enumerate(digitized)])
    
```

pole_angle 변수의 구간을 -0.5 ~ 0.5 까지 설정한 이유는 단위가 라디안 radian 이기 때문이다.

$0.5rad  = 0.5 \cdot \frac{180}{\pi} \approx 28.65$  약 29도가 된다.

digitize_state 함수의 반환값은 이산값으로 변환한 상태를 나타내는 4개의 변수를 모두 합쳐 0 부터 1295 사이의 값으로 변환한 것이다. NUM_DIGITIZED = 6 이라면 6진수로 계산한다. 만약 digitized = [[1],[2],[3],[4]] 라면 다음과 같이 변환할 수 있다.

$$ 1\cdot 6^{0} + 2\cdot 6^{1} + 3\cdot 6^{2} + 4\cdot 6^{3} = 985$$
이는 digitized  = $(\mathbb{Z}_{6})^{4}$ 와 $(0, 1295) \subset \mathbb{Z}$ 두 집합 간의 ***bijection***이 존재하기 때문에 가능하다

In [23]:
sum([x*(6**i) for i, x in enumerate([np.array([5]),np.array([5]), np.array([5]), np.array([5])])])

array([1295])

In [ ]:
help(enumerate)

#  enumerate is useful for obtaining an indexed list:
#        (0, seq[0]), (1, seq[1]), (2, seq[2]), ...

In [ ]:
help(np.digitize)
# digitize(x, bins, right=False)
#     Return the indices of the bins to which each value in input array belongs.

#     =========  =============  ============================
#     `right`    order of bins  returned index `i` satisfies
#     =========  =============  ============================
#     ``False``  increasing     ``bins[i-1] <= x < bins[i]``
#     ``True``   increasing     ``bins[i-1] < x <= bins[i]``
#     ``False``  decreasing     ``bins[i-1] > x >= bins[i]``
#     ``True``   decreasing     ``bins[i-1] >= x > bins[i]``
#     =========  =============  ============================

#     If values in `x` are beyond the bounds of `bins`, 0 or ``len(bins)`` is
#     returned as appropriate.

In [ ]:
ENV = 'CartPole-v0' # 태스크 이름
NUM_DIZITIZED = 6 # 각 상태를 이산변수로 변환할 구간 수

env = gym.make(ENV) # 실행할 태스크 설정
observation = env.reset() # 환경 초기화

In [ ]:
# 이산값으로 만들 구간 계산
def bins(climp_min, climp_max, num):
  '''
  관측된 상태(연속값)을 이산값으로 변환하는 구간을 계산
  '''
  return np.linspace(climp_min, climp_max, num+1)[1:-1]

def digitize_state(observation):
  '''
  관측된 상태(observation 변수)를 이산값으로 변환
  '''
  cart_pos, cart_v, pole_angle, pole_v = observation

  digitized = [
    np.digitize(cart_post, bins=bins(-2.4, 2.4, NUM_DIGITIZED)),
    np.digitize(cart_v, bins=bins(-3.0, 3.0, NUM_DIGITIZED)),
    np.digitize(pole_angle, bins=bins(-0.5, 0.5, NUM_DIGITIZED)),
    np.digitize(pole_v, bins=bins(-2.0, 2.0, NUM_DIGITIZED))]

  return sum([x * (NUM_DIGITIZED ** i) for i, x in enumerate(digitized)])

## Q러닝 구현